<a href="https://colab.research.google.com/github/guillegrc/TFM/blob/main/TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os, getpass
os.environ['GH_TOKEN'] = getpass.getpass('🔑 GitHub token: ')

!git clone https://$GH_TOKEN@github.com/guillegrc/TFM.git
%cd TFM

!git config --global user.email "ggarciama.inf@upsa.es"
!git config --global user.name  "guillegrc"


🔑 GitHub token: ··········
Cloning into 'TFM'...
/content/TFM


In [11]:
# ───────────────────────────────
# 0. Preparativos
# ───────────────────────────────

import gc
import time
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Reproducibilidad
SEED = 42
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()



In [8]:
# ───────────────────────────────
# 1. Datos
# ───────────────────────────────

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalizamos [0,1]
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Convertimos las etiquetas a una codificación one-hot
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

print("Train shape:", X_train.shape, y_train.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Train shape: (50000, 32, 32, 3) (50000, 10)


In [12]:
def build_cnn(n_filters=56, kernel_size=3, dropout_rate=0.15, dense_units=128, input_shape=X_train.shape[1:]):
    """
    Esta función construye un modelo base de red neuronal convolucional (CNN) con parámetros ajustables.
    - n_filters: número de filtros en las capas convolucionales.
    - kernel_size: tamaño del kernel en las capas convolucionales.
    - dropout_rate: tasa de dropout para evitar el sobreajuste.
    - dense_units: número de unidades en la capa densa final.
    """
    model = Sequential()

    # Primera capa convolucional (filtros=32, tamaño de kernel=3x3, Relu como activación)
    model.add(Conv2D(n_filters, kernel_size, padding='same',activation='relu', input_shape=X_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Conv2D(n_filters, kernel_size, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2))
    model.add(Dropout(dropout_rate))

    # Segunda capa convolucional (filtros=64)
    model.add(Conv2D(2*n_filters, kernel_size, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=2*n_filters, kernel_size=kernel_size, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2))
    model.add(Dropout(dropout_rate))

    # Capa Flatten: transforma la salida 2D de la última capa de pooling a un vector 1D
    model.add(Flatten())

    # Capa densa con unidades ajustables y activación ReLU
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Capa de salida con softmax (10 clases)
    model.add(Dense(10, activation='softmax'))

    return model

model = build_cnn()
model.summary()  # Muestra el resumen de la arquitectura de la CNN

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 56)     │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 56)     │           224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 56)     │        28,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 56)     │           224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 56)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 15, 56)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 112)    │        56,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 15, 15, 112)    │           448 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 112)    │       113,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 112)    │           448 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 112)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 112)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4032)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       516,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 718,274 (2.74 MB)

 Trainable params: 717,602 (2.74 MB)

 Non-trainable params: 672 (2.62 KB)

In [14]:
# ───────────────────────────────
# 3. Entrenamiento (clean)
# ───────────────────────────────
BATCH  = 64
EPOCHS = 50         # entrenará menos si se activa early-stopping
PATIENCE = 5


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cb = EarlyStopping(monitor='val_accuracy',
                             patience=PATIENCE,
                             restore_best_weights=True)

history = model.fit(X_train, y_train,
                    batch_size=BATCH,
                    epochs=EPOCHS,
                    validation_split=0.1,
                    callbacks=[cb],
                    verbose=1)

# # Entrenamos el modelo durante 2 épocas con un tamaño de batch de 64
# history = model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.1, verbose=1)

clean_acc = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy clean = {clean_acc:.4f}")

# ───────────────────────────────
# 4. Guardamos artefactos
# ───────────────────────────────
model.save("cnn_clean.keras")

# history → CSV (valioso para gráficas posteriores)
hist_df = pd.DataFrame(history.history)
hist_df.to_csv("history__cnn_clean.csv", index=False)


Epoch 1/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 24s 23ms/step - accuracy: 0.4051 - loss: 1.7737 - val_accuracy: 0.5814 - val_loss: 1.1717
Epoch 2/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.6325 - loss: 1.0429 - val_accuracy: 0.6854 - val_loss: 0.8893
Epoch 3/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.7049 - loss: 0.8420 - val_accuracy: 0.6448 - val_loss: 1.0739
Epoch 4/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.7431 - loss: 0.7289 - val_accuracy: 0.7366 - val_loss: 0.7785
Epoch 5/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.7822 - loss: 0.6173 - val_accuracy: 0.7290 - val_loss: 0.8198
Epoch 6/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.8130 - loss: 0.5313 - val_accuracy: 0.7568 - val_loss: 0.7242
Epoch 7/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.8304 - loss: 0.4719 - val_accuracy: 0.7750 - val_loss: 0.7129
Epoch 8/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.8518 - loss: 0.4090 - 

In [16]:
!git add history_clean.csv cnn_clean.keras
!git commit -m "Baseline CNN clean + metrics"
!git push origin main



[main (root-commit) 9d87d2d] Baseline CNN clean + metrics
 2 files changed, 28 insertions(+)
 create mode 100644 cnn_clean.keras
 create mode 100644 history_clean.csv
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 7.59 MiB | 10.47 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/guillegrc/TFM.git
 * [new branch]      main -> main
